In [71]:
import pandas as pd
import requests
from sec_api import *
import csv

In [122]:
from sec_api import ExecCompApi
def searched_names(cik):
    names = set()
    api_key = "4a56f79dfc7ec7ce841e85eb8231916dd623e3dfb2f16d7f988f5419501d8f3f"
    search_size = 50
    start_pos = 0
    
    search_params = {
      "query": f"issuer.cik:{cik}",
      "from": f'{start_pos}',
      "size": "50",
      "periodOfReport":"[2023-01-01 TO 2024-05-10]",
      "sort": [{ "filedAt": { "order": "desc" } }]
    }
    from sec_api import InsiderTradingApi
    
    insiderTradingApi = InsiderTradingApi(api_key)
    data = insiderTradingApi.get_data(search_params)
    for d in data['transactions']:
        if "name" in d['reportingOwner'].keys():
            names.add(d['reportingOwner']['name'])
    
    while len(data['transactions']) > 0:
        search_params['from'] = f'{start_pos+search_size}'
        data = insiderTradingApi.get_data(search_params)
        for d in data['transactions']:
            if "name" in d['reportingOwner'].keys():
                names.add(d['reportingOwner']['name'])
        start_pos += search_size
        
    return names

In [123]:
names = searched_names("1374328")
print(names)

{'Yakatan Seth', 'Adele Stephen D.', 'Vicis Capital, LLC', 'LANDERS ELORIAN CONNARD', 'Ordal Todd', 'Kinnaman Susan', 'JAFFE LEWIS', 'Greenblatt Jeffrey N.', 'Aramouni Fadi M.', 'ABRAMS MICHAEL S', 'ADELMAN JASON T', 'LANDOW SCOTT D', 'York Jakob', 'Ryan Patrick John Phillip', 'WILSON JOHN S', 'Dayton Judd', 'Dawson Grant Robert'}


In [124]:
cik_to_names = []
full_names = {}
with open('partial_names.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        if row[0] == "last_name":
            continue
        print(count)
        company_cik = row[2]
        last_name = row[0]
        if company_cik not in full_names:
            names = searched_names(company_cik)
            full_names[company_cik] = list(names)
        else:
            names = full_names[company_cik]
         
        found_name = ""
        for name in names:
            if last_name.lower() in name.lower() and len(last_name) > 0:
                found_name = name
                break
        cik_to_names.append([found_name, company_cik, row[3], row[1]])
        
        count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229


In [126]:
with open('full_names.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for row in cik_to_names:
        writer.writerow(row)